# 📚 ДЗ №2: Работа с данными для LLM

## 🎯 Цель задания
После выполнения задания вы сможете:
- Предобрабатывать русскоязычные текстовые данные для LLM
- Работать с готовыми моделями HuggingFace для анализа тональности и NER
- Создавать эффективные промпты для LLM API
- Сравнивать качество работы разных подходов к анализу текста
- Формировать датасеты в формате instruction-following для fine-tuning
- Сохранять данные в правильных форматах для обучения LLM

## 📝 Структура задания
- **Часть 1** (35% оценки): Предобработка данных и работа с готовыми моделями
- **Часть 2** (35% оценки): LLM API и prompt engineering
- **Часть 3** (20% оценки): Подготовка данных для fine-tuning LLM
- **Часть 4** (10% оценки): Сравнительный анализ и визуализация

## ⚡ Критерии оценки
- Качество предобработки данных: 25%
- Корректность работы с готовыми моделями: 20%
- Эффективность промптов для LLM: 25%
- Правильность подготовки данных для fine-tuning: 20%
- Качество сравнительного анализа: 10%


## 🔧 Установка зависимостей

Установим необходимые библиотеки для работы с данными, готовыми моделями и LLM API.


In [1]:
%pip install pandas numpy matplotlib seaborn
%pip install transformers torch
%pip install openai>=1.0.0  # Для работы с OpenAI API
%pip install datasets
%pip install pymorphy2



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
zsh:1: 1.0.0 not found
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 4.9 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 6.7 MB/s  0:00:066m0:00:0100:01
  Attempting uninstall: fsspecm━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  3/13 [multidict]
    Found existing installation: fsspec 2025.7.0━━━━━━━━━━━━━━  3/13 [multidict]
    Uninstalling fsspec-2025.7.0:━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  3/13 [multidict]
      Successfully uninstalled fsspec-2025.7.0━━━━━━━━━━━━━━━━  3/13 [multidict]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13/13 [datasets]/13 [datasets]ess]
Note: you may need to restart the kernel to use updated packages.
  Using cached docopt-0.6.2-py2.py3-none-any.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 10.5 MB/s  0:00:00 eta 0:00:01
  

In [2]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from typing import List, Dict, Tuple
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
import warnings
warnings.filterwarnings('ignore')

# Настройка отображения
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

print("Библиотеки загружены успешно!")



Библиотеки загружены успешно!


## 📊 Часть 1: Предобработка данных и готовые модели (35% оценки)

### Задание 1.1: Анализ "грязного" датасета

Проанализируем реалистичный датасет с типичными проблемами: опечатки, разные регистры, лишние пробелы, эмодзи.


In [13]:
# Создаем "грязный" датасет с типичными проблемами реальных данных
# Включаем сложные случаи для демонстрации преимуществ LLM
raw_reviews = [
    # Простые случаи
    "отличный iphone 14 PRO!!!  купил в магазине  apple на тверской 😊. Камера супер#positive",
    "УЖАСНОЕ обслуживание в сбербанке на красной площади.. менеджер иван петров вобще не помог(#negative",

    # Сарказм и ирония (сложно для классических моделей)
    "Спасибо огромное сотрудникам МТС за то что 3 часа держали меня в очереди! Просто восхитительно 👏#negative",
    "Какой замечательный сервис в Пятерочке - касса сломалась прямо передо мной, а персонал даже не извинился#negative",

    # Смешанные эмоции
    "iPhone 13 хороший телефон, но цена кусается. В целом доволен покупкой в re:Store#neutral",
    "Ресторан Белуга красивый и атмосфера приятная, но официант Максим был невнимателен#negative",

    # Сложная структура предложений
    "Хотя Tesla Model Y и дорогая машина, и сервис в Рольф Премиум иногда подводит, но в целом я очень доволен покупкой#positive",
    "Не могу сказать что отель Ритц-Карлтон плохой, просто ожидал большего за такие деньги#neutral",

    # Контекстно-зависимые случаи
    "Заказал доставку в Яндекс.Еде из ресторана Дача на Рублевке - привезли холодное, но курьер Андрей был вежливый#positive",
    "MacBook Pro 16 работает как часы уже год, покупал в iStore на Арбате у консультанта Елены#positive",

    # Неоднозначные случаи
    "Сходил в кинотеатр Октябрь посмотреть новый фильм Marvel - ну такое себе, но попкорн вкусный был#neutral",
    "Обслуживание в банке ВТБ на Тверской оставляет желать лучшего, хотя менеджер Ольга старалась помочь#negative",

    # Сложные именованные сущности
    "Купил новый Samsung Galaxy S24 Ultra в DNS на Ленинском проспекте, консультант Дмитрий Иванович всё объяснил#neutral",
    "Ужинал в ресторане White Rabbit на Смоленской площади - шеф-повар Владимир Мухин превзошел ожидания#positive",

    # Опечатки и сленг
    "норм телек LG купил в эльдорадо, продавец норм чел был, всё рассказал про функции#positive"
]

# TODO: Создайте DataFrame и проанализируйте проблемы в данных
# Создайте DataFrame из списка raw_reviews
# Добавьте колонку с правильными метками тональности для каждого отзыва
# Проанализируйте и выведите список проблем, которые вы видите в данных

# Проблемы:
#     - повторяющиеся знаки препинания
#     - ошибки
#     - эмодзи
#     - названия с использованием знаков Яндекс.Еде, re:Store
#     - эмоции через знаки

# Подумайте: какие проблемы могут повлиять на качество анализа?

# Ваш код здесь:

df = pd.DataFrame()
df['text'] = None
df['mark'] = None

for review in raw_reviews:

    review_text, review_mark = review.split("#")

    df = pd.concat([df, pd.DataFrame({'text': [review_text], 'mark': [review_mark]})], ignore_index=True)


# Анализ структуры данных
print("Структура данных:")
print(f"Количество отзывов: {df.shape[0]}")
print(f"Колонки: {list(df.columns)}")

print("\nДлина текстов отзывов:")
df["text_length"] = df["text"].str.len()
print(f"Средняя длина: {df['text_length'].mean():.1f} символов")
print(f"Минимальная: {df['text_length'].min()} символов")
print(f"Максимальная: {df['text_length'].max()} символов")

print(f"Загружено {df.shape[0]} отзывов из общего списка:")
df.head()



Структура данных:
Количество отзывов: 15
Колонки: ['text', 'mark']

Длина текстов отзывов:
Средняя длина: 94.1 символов
Минимальная: 78 символов
Максимальная: 114 символов
Загружено 15 отзывов из общего списка:


,text,mark,text_length
0,отличный iphone 14 PRO!!! купил в магазине a...,positive,78
1,УЖАСНОЕ обслуживание в сбербанке на красной пл...,negative,90
2,Спасибо огромное сотрудникам МТС за то что 3 ч...,negative,96
3,Какой замечательный сервис в Пятерочке - касса...,negative,104
4,"iPhone 13 хороший телефон, но цена кусается. В...",neutral,80


### Задание 1.2: Очистка и нормализация данных


In [15]:
def clean_text(text: str) -> str:
    if not text or pd.isna(text):
        return ""

    text = re.sub(r"[^\w\s\-.,!?;:()]", "", text)

    text = re.sub(r"\s+", " ", text)
    text = text.strip()

    text = re.sub(r"[.]{2,}", ".", text)
    text = re.sub(r"[!]{2,}", "!", text)
    text = re.sub(r"[?]{2,}", "?", text)

    if text:
        text = text[0].upper() + text[1:].lower()

    return text

df["cleaned_text"] = df["text"].apply(clean_text)

print(f"\nСредняя длина до очистки: {df['text'].str.len().mean():.1f}")
print(f"Средняя длина после очистки: {df['cleaned_text'].str.len().mean():.1f}")



Средняя длина до очистки: 94.1
Средняя длина после очистки: 93.5


### Задание 1.3: Использование готовых моделей HuggingFace


In [30]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# TODO: Загрузите готовые модели HuggingFace для анализа тональности и NER
# Исследуйте HuggingFace Hub и найдите подходящие русскоязычные модели для:
# - Анализа тональности (sentiment analysis)
# - Извлечения именованных сущностей (NER)
#
# Используйте функцию pipeline() из библиотеки transformers
# Обратите внимание на параметры модели и токенизатора

# Ваш код для загрузки моделей:

def analyze_with_huggingface(texts: List[str]) -> List[Dict]:
    """
    Анализ текстов с помощью готовых моделей HuggingFace
    """
    # TODO: Реализуйте функцию анализа
    # Для каждого текста:
    # 1. Примените модель анализа тональности
    # 2. Примените модель NER
    # 3. Соберите результаты в структурированном виде
    # 4. Верните список словарей с результатами

    sentiment_model = pipeline(
        "sentiment-analysis",
        model="blanchefort/rubert-base-cased-sentiment-rusentiment",
        return_all_scores=True
    )

    ner_model_id = "r1char9/ner-rubert-tiny-RuNews"

    label2id = {
        'O': 0,
        'B-GEOPOLIT': 1, 'I-GEOPOLIT': 2,
        'B-MEDIA': 3,    'I-MEDIA': 4,
        'B-LOC': 5,      'I-LOC': 6,
        'B-ORG': 7,      'I-ORG': 8,
        'B-PER': 9,      'I-PER': 10
    }
    id2label = {v: k for k, v in label2id.items()}

    ner_model = AutoModelForTokenClassification.from_pretrained(
        ner_model_id,
        num_labels=len(label2id),
        id2label=id2label,
        label2id=label2id
    )

    ner_pipeline = pipeline(
        "ner",
        model=ner_model,
        tokenizer=AutoTokenizer.from_pretrained(ner_model_id),
        aggregation_strategy="simple"
    )

    label_mapping = {
        "POSITIVE": "positive",
        "NEGATIVE": "negative",
        "NEUTRAL": "neutral"
    }

    for text in texts:

        result = sentiment_model(text)

        print(f"Входной текст: {text}")

        if result and len(result[0]) > 0:
            best_sentiment = max(result[0], key=lambda x: x["score"])

            sentiment = label_mapping.get(best_sentiment["label"].upper(), best_sentiment["label"].lower())

            print(f"Тональность: {sentiment}")

            ner_entity = ner_pipeline(text)
            entity = []

            for ner in ner_entity:
                entity.append(ner['word'])

            print(f"Найденные сущности: {entity}")


analyze_with_huggingface(df["cleaned_text"])



Device set to use cpu
Device set to use cpu


Входной текст: Отличный iphone 14 pro! купил в магазине apple на тверской . камера супер
Тональность: positive
Найденные сущности: []
Входной текст: Ужасное обслуживание в сбербанке на красной площади. менеджер иван петров вобще не помог(
Тональность: negative
Найденные сущности: []
Входной текст: Спасибо огромное сотрудникам мтс за то что 3 часа держали меня в очереди! просто восхитительно
Тональность: positive
Найденные сущности: []
Входной текст: Какой замечательный сервис в пятерочке - касса сломалась прямо передо мной, а персонал даже не извинился
Тональность: negative
Найденные сущности: []
Входной текст: Iphone 13 хороший телефон, но цена кусается. в целом доволен покупкой в re:store
Тональность: neutral
Найденные сущности: []
Входной текст: Ресторан белуга красивый и атмосфера приятная, но официант максим был невнимателен
Тональность: negative
Найденные сущности: []
Входной текст: Хотя tesla model y и дорогая машина, и сервис в рольф премиум иногда подводит, но в целом я очень 

## 🤖 Часть 2: LLM API и Prompt Engineering (35% оценки)

### Задание 2.1: Создание эффективных промптов


In [ ]:
def create_prompts_for_llm() -> Dict[str, str]:
    """
    Создание базовых промптов для разных задач (один промпт на задачу)
    """
    # TODO: Создайте эффективные промпты для NER и sentiment analysis
    # Подумайте о структуре хорошего промпта:
    # - Четкое описание задачи
    # - Примеры входных и выходных данных
    # - Формат ответа (JSON, текст и т.д.)
    # - Особые требования (например, для русского языка)

    # Создайте промпты для:
    # 1. Извлечения именованных сущностей (NER)
    # 2. Анализа тональности (sentiment analysis)

    # Ваш код здесь:
    pass

# TODO: Протестируйте ваши промпты
# Выведите созданные промпты и оцените их качество

# TODO: Настройте OpenAI API
# Установите API ключ через переменные окружения
# Изучите документацию OpenAI API для Python



# TODO: Реализуйте функции для работы с OpenAI API
# Создайте функции для:
# 1. Вызова OpenAI API с промптом
# 2. Обработки ответа от API
# 3. Анализа текстов с помощью ваших промптов
#
# Подумайте о:
# - Обработке ошибок API
# - Формате запроса и ответа
# - Параметрах модели (temperature, max_tokens)
#
# Протестируйте на нескольких текстах из датасета



### Задание 2.2: Сравнение результатов HuggingFace vs LLM


In [ ]:
# TODO: Сравните результаты HuggingFace моделей с LLM на одних и тех же текстах
# Создайте сравнительный анализ:
# 1. Соберите результаты обеих подходов в структурированном виде
# 2. Сравните точность анализа тональности
# 3. Сравните качество извлечения сущностей
# 4. Проанализируйте время выполнения
# 5. Оцените простоту использования
#
# Создайте визуализации для сравнения:
# - Точность по разным метрикам
# - Время обработки
# - Количество найденных сущностей
#
# Сделайте выводы о том, когда лучше использовать каждый подход



## 📚 Часть 3: Подготовка данных для Fine-tuning LLM (20% оценки)

### Задание 3.1: Создание instruction-following датасета


In [ ]:
### Задание 2.3: Анализ сложных случаев

# Выберем специально сложные примеры для демонстрации преимуществ LLM
complex_cases = [
    "Спасибо огромное сотрудникам МТС за то что 3 часа держали меня в очереди! Просто восхитительно 👏",
    "iPhone 13 хороший телефон, но цена кусается. В целом доволен покупкой в re:Store",
    "Хотя Tesla Model Y и дорогая машина, и сервис в Рольф Премиум иногда подводит, но в целом я очень доволен покупкой",
    "норм телек LG купил в эльдорадо, продавец норм чел был, всё рассказал про функции"
]

print("Анализ сложных случаев:")
print("=" * 60)

# TODO: Сравните результаты HuggingFace и OpenAI на сложных случаях
# for i, text in enumerate(complex_cases):
#     print(f"\nПример {i+1}: {text}")
#     # hf_result = sentiment_pipeline(text)
#     # openai_result = analyze_with_openai([text])
#     # print(f"HuggingFace: {hf_result}")
#     # print(f"OpenAI: {openai_result}")





In [ ]:
### Задание 2.4: Количественное сравнение точности

# Создаем расширенный набор для тестирования с правильными ответами
test_cases_with_labels = [
    # Сарказм и ирония - должны быть NEGATIVE
    ("Спасибо огромное сотрудникам МТС за то что 3 часа держали меня в очереди! Просто восхитительно 👏", "NEGATIVE"),
    ("Какой замечательный сервис в Пятерочке - касса сломалась прямо передо мной, а персонал даже не извинился", "NEGATIVE"),

    # Смешанные эмоции - должны быть NEUTRAL или зависеть от преобладающего тона
    ("iPhone 13 хороший телефон, но цена кусается. В целом доволен покупкой в re:Store", "NEUTRAL"),
    ("Ресторан Белуга красивый и атмосфера приятная, но официант Максим был невнимателен", "NEUTRAL"),

    # Сложные структуры - требуют понимания контекста
    ("Хотя Tesla Model Y и дорогая машина, и сервис в Рольф Премиум иногда подводит, но в целом я очень доволен покупкой", "POSITIVE"),
    ("Не могу сказать что отель Ритц-Карлтон плохой, просто ожидал большего за такие деньги", "NEUTRAL"),

    # Неформальная речь и сленг
    ("норм телек LG купил в эльдорадо, продавец норм чел был, всё рассказал про функции", "POSITIVE"),
    ("Сходил в кинотеатр Октябрь посмотреть новый фильм Marvel - ну такое себе, но попкорн вкусный был", "NEUTRAL"),

    # Простые случаи для контроля
    ("отличный iphone 14 PRO!!! купил в магазине apple на тверской 😊. Камера супер", "POSITIVE"),
    ("УЖАСНОЕ обслуживание в сбербанке на красной площади.. менеджер иван петров вобще не помог(", "NEGATIVE")
]

# TODO: Рассчитайте точность для каждой модели
# hf_correct = 0
# openai_correct = 0
# total = len(test_cases_with_labels)



In [ ]:
### Задание 2.5: Визуализация сравнения моделей

import matplotlib.pyplot as plt
import numpy as np

# TODO: Создайте визуализацию сравнения точности моделей
# plt.figure(figsize=(12, 8))
# # Создайте графики сравнения

In [ ]:
def create_instruction_dataset(df: pd.DataFrame) -> List[Dict]:
    """
    Создание датасета в формате instruction-following для fine-tuning LLM
    """
    # TODO: Создайте структурированный датасет для fine-tuning LLM
    # Подумайте о структуре instruction-following датасета:
    # - Какие поля должны быть в каждом примере?
    # - Как сформулировать инструкции для модели?
    # - Какие типы задач включить (sentiment, NER, etc.)?
    # - Как структурировать ответы модели?
    #
    # Создайте несколько примеров для разных задач

    pass

# TODO: Протестируйте созданный датасет
# Создайте и проанализируйте instruction dataset
# Выведите примеры в читаемом формате
# Проанализируйте распределение типов задач



### Задание 3.2: Сериализация данных в формате для LLM платформ


In [ ]:
import json

# TODO: Реализуйте сохранение данных в форматах для fine-tuning
# Создайте функции для сохранения данных в форматах:
# 1. JSONL формат для OpenAI fine-tuning API
# 2. CSV формат для общего использования
#
# Изучите требования к форматам:
# - Какая структура нужна для OpenAI fine-tuning?
# - Как правильно структурировать messages?
# - Какие поля обязательны?
#
# Протестируйте сохранение и загрузку данных


